In [2]:
import os
import torch
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import torch.nn as nn
import torchvision.models as models
from torch.autograd import Variable
import pandas as pd


data_dir = os.path.join("./", "TrainTest")

In [3]:
resnet18 = models.resnet18(pretrained=True)
modules=list(resnet18.children())[:-1]
resnet18=nn.Sequential(*modules)
for p in resnet18.parameters():
    p.requires_grad = False;

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

image_features = np.zeros((513, 1))

resnet18.eval()
 
with torch.no_grad():
    for data in dataloader: 
        image, label = data
        encoding = resnet18(image) 

        encoding_np = encoding.numpy()
        label = label.numpy()

        encoding_np = encoding_np.reshape(1, 512)
        encoding_np = np.append(encoding_np, label)
        encoding_np = encoding_np.reshape(513, 1)

        image_features = np.append(image_features, encoding_np, axis=1)
        

        

# drop first row of zeros
image_features_df = pd.DataFrame(image_features)
image_features_df = image_features_df.iloc[:, 1:]
print(image_features_df)

         1         2         3         4         5         6         7     \
0    0.271028  0.431392  0.189157  0.181322  0.557896  0.397118  0.978484   
1    0.208780  0.269095  0.248151  0.431159  0.148965  0.217441  0.109632   
2    0.152388  0.121084  0.175657  0.183613  0.297226  0.328614  0.289818   
3    1.255196  1.369295  1.353558  1.435808  1.440755  1.390106  1.360685   
4    0.453387  0.165390  0.167985  0.244700  0.265388  0.243067  0.192398   
..        ...       ...       ...       ...       ...       ...       ...   
508  0.308012  0.352928  0.213455  0.158261  0.234877  0.336584  0.409896   
509  1.168912  1.118457  0.870525  1.115834  1.566571  1.275557  1.559748   
510  0.035456  0.044374  0.115242  0.108024  0.074112  0.043260  0.099156   
511  0.235910  0.135839  0.084740  0.225783  0.155174  0.068174  0.044899   
512  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

         8         9         10    ...      1946      1947      1948  \
0  

In [15]:
image_features_df_T = image_features_df.T
image_features_df_T = pd.DataFrame(image_features_df_T)
labels = image_features_df_T.iloc[:, 512]
print(labels.shape)
print(image_features_df_T.shape)


(1955,)
(1955, 513)


In [16]:
feature_set_x = image_features_df_T.iloc[:, :512]
print(feature_set_x.shape)

(1955, 512)


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

X_train, X_test, y_train, y_test = train_test_split(feature_set_x, labels, stratify = labels, test_size=0.3)

model = LogisticRegressionCV()
model.fit(X_train, y_train)
s = model.score(X_test, y_test)
print("The score of the model is: ", s)

C:\Users\grace\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\grace\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

The score of the model is:  0.9403747870528109
